In [ ]:
import pandas as pd

FILE = "/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv"
SIZE = 10000
VIRAL_THRESHOLD = 100

data = pd.read_csv(FILE)
data = data.sample(SIZE)

In [ ]:
data = data[['title', 'num_points']]

In [ ]:
viral = data["num_points"].apply(lambda x: pd.Series(1 if x >= VIRAL_THRESHOLD else 0))
viral.columns=["viral"]

In [ ]:
#skew checking
viral.describe()

In [ ]:
from nltk.tokenize import StanfordTokenizer
import numpy as np
title = data["title"].apply(lambda x: pd.Series(StanfordTokenizer().tokenize(x)))
title = title.applymap(lambda x: '.' if x is np.nan else x.lower())

In [ ]:
import DataLoader.GloVe as gl
glove = gl.load('/Users/Shared/data/glove.6B/glove.6B.50d.txt')

In [ ]:
def to_word_vec(word, embedding):
    try:
        return embedding.loc[word]
    except:
        return embedding.loc['.']

title_emb = []
for i in title.columns:
    title_emb.append(title[i].apply(lambda x: to_word_vec(x, glove)))
    title_emb[-1].columns = [ i * glove.shape[1] + j for j in title_emb[i].columns]

In [ ]:
hn_data = viral
for i in range(len(title_emb)):
    hn_data = hn_data.join(title_emb[i])

In [ ]:
hn_data.to_csv('./hn_gloves_%d_%d' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)

In [ ]:
testing_data = hn_data.sample(frac=0.2)
training_data = hn_data.drop(testing_data.index)

testing_data.to_csv('./hn_testing_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)
training_data.to_csv('./hn_training_%d_%d.csv' % (SIZE, VIRAL_THRESHOLD), header=False, index=False)